In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', 500)


In [2]:
def score(tail_cond, tail_marg, head_cond, head_marg, lam = 1):
    return ((lam * (tail_cond + head_cond) - (tail_marg + head_marg)))/2

In [14]:
df_tuples = pd.read_csv(
    '../FilesToUploadToColab/NovelTuples.csv', 
    sep = '\t',
    header = None
)
df_tuples.columns = ('rel','head','tail','misc','misc2')

df_results_no_grammar = pd.read_csv(
    '../FilesToUploadToColab/wikipedia_predictions_template_no_grammar.csv'
)
df_results_grammar = pd.read_csv(
    '../FilesToUploadToColab/wikipedia_predictions_template_grammar.csv'
)

df_wikipedia_final = pd.read_csv(
    '../FilesToUploadToColab/wikipedia_top_final.csv'
)
df_wikipedia_final = df_wikipedia_final[[
    'rel', 
    'head', 
    'tail',
    'Josh-Rating',
    'Joe-Rating',
    'Avg-Rating'
]]

In [15]:
for lam in np.arange(1, 11, 1):
    df_tuples[f'score_grammar_{lam}'] = score(
        df_results_grammar.tail_conditional,
        df_results_grammar.tail_marginal, 
        df_results_grammar.head_conditional, 
        df_results_grammar.head_marginal, 
        lam = lam
    )
    df_tuples[f'score_no_grammar_{lam}'] = score(
        df_results_no_grammar.tail_conditional,
        df_results_no_grammar.tail_marginal, 
        df_results_no_grammar.head_conditional, 
        df_results_no_grammar.head_marginal, 
        lam = lam
    )

In [16]:
df_tuples['top_100'] = False

In [17]:
for clm in df_tuples.columns:
    if 'score' in clm:
        df_tuples[clm.replace('score', 'rank')] = df_tuples[clm].rank(ascending=False)        

In [18]:
for clm in df_tuples.columns:
    if 'score' in clm:
        df_tuples.loc[df_tuples[clm].rank(ascending=False) <= 100, 'top_100'] = True

In [19]:
for clm in df_tuples.columns:
    if 'score' in clm:
        df_tuples[clm.replace('score', 'rank')] = df_tuples[clm].rank(ascending=False)        

In [20]:
df_tuples = df_tuples.merge(df_wikipedia_final, how = 'left',on = ['rel', 'head', 'tail'])

In [23]:
df_tuples = df_tuples[df_tuples.top_100]

In [25]:
df_tuples.to_csv('ablations.csv')